In [1]:
import sys
import os
import ast
from snowflake.nodes import *
from snowflake.stencil_compiler import PythonCompiler, CCompiler
from ctree.frontend import dump
import numpy as np
import inspect
!ctree -cc

ctree looking for relative cache directories named compiled, checking directories under this one



In [2]:
compiler = PythonCompiler()

In [3]:
arr = WeightArray([
    [1/9.0, 1/9.0, 1/9.0],
    [1/9.0, 1/9.0, 1/9.0],
    [1/9.0, 1/9.0, 1/9.0]
])
#arr2 = WeightArray([0])
s = StencilComponent("a", arr)
simple_stencil = Stencil(s, "out", ((1, -1, 1), (1, -1, 1)))
size = 34

In [4]:
output = np.zeros((size, size), dtype=np.double)

In [5]:
output

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [6]:
kernel = compiler.compile(simple_stencil)

In [7]:
test_data = np.arange(size*size, dtype=np.double).reshape(size, size)

In [8]:
kernel(output, test_data)

In [9]:
output

array([[    0.,     0.,     0., ...,     0.,     0.,     0.],
       [    0.,    35.,    36., ...,    65.,    66.,     0.],
       [    0.,    69.,    70., ...,    99.,   100.,     0.],
       ..., 
       [    0.,  1055.,  1056., ...,  1085.,  1086.,     0.],
       [    0.,  1089.,  1090., ...,  1119.,  1120.,     0.],
       [    0.,     0.,     0., ...,     0.,     0.,     0.]])

In [10]:
test_data

array([[  0.00000000e+00,   1.00000000e+00,   2.00000000e+00, ...,
          3.10000000e+01,   3.20000000e+01,   3.30000000e+01],
       [  3.40000000e+01,   3.50000000e+01,   3.60000000e+01, ...,
          6.50000000e+01,   6.60000000e+01,   6.70000000e+01],
       [  6.80000000e+01,   6.90000000e+01,   7.00000000e+01, ...,
          9.90000000e+01,   1.00000000e+02,   1.01000000e+02],
       ..., 
       [  1.05400000e+03,   1.05500000e+03,   1.05600000e+03, ...,
          1.08500000e+03,   1.08600000e+03,   1.08700000e+03],
       [  1.08800000e+03,   1.08900000e+03,   1.09000000e+03, ...,
          1.11900000e+03,   1.12000000e+03,   1.12100000e+03],
       [  1.12200000e+03,   1.12300000e+03,   1.12400000e+03, ...,
          1.15300000e+03,   1.15400000e+03,   1.15500000e+03]])

In [11]:
ccompiler = CCompiler()

In [12]:
ckernel = ccompiler.compile(simple_stencil)

In [13]:
output[:] = 0
ckernel(output, test_data)

In [14]:
output

array([[    0.,     0.,     0., ...,     0.,     0.,     0.],
       [    0.,     0.,     0., ...,     0.,     0.,     0.],
       [    0.,     0.,    70., ...,    99.,   100.,     0.],
       ..., 
       [    0.,     0.,  1056., ...,  1085.,  1086.,     0.],
       [    0.,     0.,  1090., ...,  1119.,  1120.,     0.],
       [    0.,     0.,     0., ...,     0.,     0.,     0.]])

In [15]:
output.shape

(34, 34)

In [17]:
output[0:4, 0:4]

array([[   0.,    0.,    0.,    0.],
       [   0.,    0.,    0.,    0.],
       [   0.,    0.,   70.,   71.],
       [   0.,    0.,  104.,  105.]])